## IMPORT DATA

In [59]:
import torch
import torch.nn.functional as F
from ogb.nodeproppred import Evaluator, PygNodePropPredDataset
from torch.nn import LayerNorm, Linear, ReLU
from tqdm import tqdm

from torch_geometric.loader import RandomNodeLoader

In [60]:
import os.path as osp
import sys

import matplotlib.pyplot as plt
import torch
from ogb.nodeproppred import Evaluator, PygNodePropPredDataset
from torch_geometric.loader import DataLoader

# Download and process data at './dataset/ogbg_molhiv/'
ogbnproteins_dataset = PygNodePropPredDataset('ogbn-proteins', root='../data')
ogbnarxiv_dataset = PygNodePropPredDataset(name='ogbn-arxiv', root= '../data')

## Prepare DataLoader

In [67]:
## DataLoader For ogbnproteins datasett
from torch_geometric.utils import scatter
from torch_geometric.loader import RandomNodeLoader
splitted_idx = ogbnproteins_dataset.get_idx_split()
data = ogbnproteins_dataset[0]

data.y = data.y.to(torch.float)

# Initialize features of nodes by aggregating edge features.
row, col = data.edge_index
data.x = scatter(data.edge_attr, col, dim_size=data.num_nodes, reduce='sum')

# Set split indices to masks.
for split in ['train', 'valid', 'test']:
    mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    mask[splitted_idx[split]] = True
    data[f'{split}_mask'] = mask
    
train_loader = RandomNodeLoader(data, num_parts=40, shuffle=True,
                                num_workers=5)
test_loader = RandomNodeLoader(data, num_parts=5, num_workers=5)

In [68]:
data.x[data.train_mask].shape,data.y[data.train_mask].shape, data.x.size(-1)

(torch.Size([86619, 8]), torch.Size([86619, 112]), 8)

In [69]:
data['train_mask']

tensor([ True,  True,  True,  ..., False, False, False])

## Prepare Models

### GCN MODEL

In [63]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from ogb.nodeproppred import Evaluator
from tqdm import tqdm

    
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv1(x, edge_index).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(data.x.size(-1), 64, data.y.size(-1)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)
criterion = torch.nn.BCEWithLogitsLoss()
evaluator = Evaluator('ogbn-proteins')


def train(epoch):
    model.train()

   

    total_loss = total_examples = 0
    for data in train_loader:
        optimizer.zero_grad()
        data = data.to(device)
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * int(data.train_mask.sum())
        total_examples += int(data.train_mask.sum())



    return total_loss / total_examples


@torch.no_grad()
def test():
    model.eval()

    y_true = {'train': [], 'valid': [], 'test': []}
    y_pred = {'train': [], 'valid': [], 'test': []}



    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index)

        for split in y_true.keys():
            mask = data[f'{split}_mask']
            y_true[split].append(data.y[mask].cpu())
            y_pred[split].append(out[mask].cpu())



    train_rocauc = evaluator.eval({
        'y_true': torch.cat(y_true['train'], dim=0),
        'y_pred': torch.cat(y_pred['train'], dim=0),
    })['rocauc']

    valid_rocauc = evaluator.eval({
        'y_true': torch.cat(y_true['valid'], dim=0),
        'y_pred': torch.cat(y_pred['valid'], dim=0),
    })['rocauc']

    test_rocauc = evaluator.eval({
        'y_true': torch.cat(y_true['test'], dim=0),
        'y_pred': torch.cat(y_pred['test'], dim=0),
    })['rocauc']

    return train_rocauc, valid_rocauc, test_rocauc


print(f'Training And Test ')
for epoch in tqdm(range(1, 101)):

    loss = train(epoch)
   
    train_rocauc, valid_rocauc, test_rocauc = test()
    print(f'Loss: {loss:.4f}, Train: {train_rocauc:.4f}, '
          f'Val: {valid_rocauc:.4f}, Test: {test_rocauc:.4f}')

Training And Test 



  1%|          | 1/100 [02:23<3:56:34, 143.38s/it]

Loss: 0.9122, Train: 0.5007, Val: 0.3939, Test: 0.4202



  2%|▏         | 2/100 [05:05<4:12:09, 154.38s/it]

Loss: 0.3953, Train: 0.5631, Val: 0.4793, Test: 0.4189


...

Loss: 0.3237, Train: 0.6534, Val: 0.6424, Test: 0.5163



 99%|█████████▉| 99/100 [14:31:22<02:58, 178.62s/it]

Loss: 0.3256, Train: 0.6605, Val: 0.6583, Test: 0.5390



100%|██████████| 100/100 [14:34:31<00:00, 524.71s/it]

Loss: 0.3252, Train: 0.6505, Val: 0.6417, Test: 0.5106


### Node2vec Model

In [133]:
import os.path as osp
import sys
from tqdm import tqdm

from torch_geometric.nn import Node2Vec
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Node2Vec(
    data.edge_index,
    embedding_dim=data.y.size(-1),
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=1.0,
    q=1.0,
    sparse=True,
).to(device)

num_workers = 4 if sys.platform == 'linux' else 0
loader = model.loader(batch_size=128, shuffle=True, num_workers=num_workers)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)



@torch.no_grad()
def test():
    model.eval()
    z = model()
    y_true = {'train': [], 'valid': [], 'test': []}
    y_pred = {'train': [], 'valid': [], 'test': []}

 
   
    for split in y_true.keys():
        mask = data[f'{split}_mask']
        y_true[split].append(data.y[mask].cpu())
        y_pred[split].append(z[mask].cpu())


    train_rocauc = evaluator.eval({
        'y_true': torch.cat(y_true['train'], dim=0),
        'y_pred': torch.cat(y_pred['train'], dim=0),
    })['rocauc']

    valid_rocauc = evaluator.eval({
        'y_true': torch.cat(y_true['valid'], dim=0),
        'y_pred': torch.cat(y_pred['valid'], dim=0),
    })['rocauc']

    test_rocauc = evaluator.eval({
        'y_true': torch.cat(y_true['test'], dim=0),
        'y_pred': torch.cat(y_pred['test'], dim=0),
    })['rocauc']

    return train_rocauc, valid_rocauc, test_rocauc




print(f'Training And Test ')
for epoch in tqdm(range(1, 21)):

    loss = train()
   
    train_rocauc, valid_rocauc, test_rocauc = test()
    print(f'Loss: {loss:.4f}, Train: {train_rocauc:.4f}, '
          f'Val: {valid_rocauc:.4f}, Test: {test_rocauc:.4f}')

Training And Test 


  5%|▌         | 1/20 [06:58<2:12:25, 418.20s/it]

Loss: 4.0124, Train: 0.4996, Val: 0.5008, Test: 0.5001


 10%|█         | 2/20 [14:42<2:13:35, 445.29s/it]

Loss: 1.2488, Train: 0.4991, Val: 0.5014, Test: 0.5002


...

Loss: 1.0424, Train: 0.4968, Val: 0.4979, Test: 0.4971


 95%|█████████▌| 19/20 [2:30:54<08:01, 481.33s/it]

Loss: 1.0423, Train: 0.4966, Val: 0.4985, Test: 0.4970


100%|██████████| 20/20 [2:38:57<00:00, 476.87s/it]

Loss: 1.0418, Train: 0.4956, Val: 0.4983, Test: 0.4969


## MLP model

In [99]:
from sklearn.neural_network import MLPClassifier
import numpy as np

X_train, X_valid, X_test = data.x[splitted_idx['train']], data.x[splitted_idx['valid']], data.x[splitted_idx['test']]

y_train, y_valid, y_test = data.y[splitted_idx['train']], data.y[splitted_idx['valid']], data.y[splitted_idx['test']]
X_train = torch.concat((X_train,X_valid))
y_train = torch.concat((y_train,y_valid))
model = MLPClassifier()
model.fit(X_train, y_train)

MLPClassifier()

In [102]:
from sklearn.metrics import roc_auc_score
ypred_train = model.predict(data.x[splitted_idx['train']])
ypred_valid = model.predict(X_valid)
ypred_test = model.predict(X_test)

In [105]:
score_train = roc_auc_score(ypred_train, data.y[splitted_idx['train']])
score_valid = roc_auc_score(ypred_valid, y_valid)
score_test = roc_auc_score(ypred_test, y_test)
print(f"Train score {score_train} , Valid score {score_valid}, Test score {score_test}")

Train score 0.789416474889874 , Valid score 0.7830178677932498, Test score 0.5429647230357008


## GraphSAGE model

In [132]:
from torch_geometric.nn import GraphSAGE
model = GraphSAGE(
    data.num_node_features,
    hidden_channels=data.y.size(-1),
    num_layers=2,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


def train(epoch):
    model.train()

   

    total_loss = total_examples = 0
    for data in train_loader:
        optimizer.zero_grad()
        data = data.to(device)
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * int(data.train_mask.sum())
        total_examples += int(data.train_mask.sum())



    return total_loss / total_examples

@torch.no_grad()
def test():
    model.eval()

    y_true = {'train': [], 'valid': [], 'test': []}
    y_pred = {'train': [], 'valid': [], 'test': []}



    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index)

        for split in y_true.keys():
            mask = data[f'{split}_mask']
            y_true[split].append(data.y[mask].cpu())
            y_pred[split].append(out[mask].cpu())



    train_rocauc = evaluator.eval({
        'y_true': torch.cat(y_true['train'], dim=0),
        'y_pred': torch.cat(y_pred['train'], dim=0),
    })['rocauc']

    valid_rocauc = evaluator.eval({
        'y_true': torch.cat(y_true['valid'], dim=0),
        'y_pred': torch.cat(y_pred['valid'], dim=0),
    })['rocauc']

    test_rocauc = evaluator.eval({
        'y_true': torch.cat(y_true['test'], dim=0),
        'y_pred': torch.cat(y_pred['test'], dim=0),
    })['rocauc']

    return train_rocauc, valid_rocauc, test_rocauc


print(f'Training And Test ')
for epoch in tqdm(range(1, 101)):

    loss = train(epoch)
   
    train_rocauc, valid_rocauc, test_rocauc = test()
    print(f'Loss: {loss:.4f}, Train: {train_rocauc:.4f}, '
          f'Val: {valid_rocauc:.4f}, Test: {test_rocauc:.4f}')

Training And Test 


  1%|          | 1/100 [01:06<1:50:31, 66.99s/it]

Loss: 2.1480, Train: 0.6647, Val: 0.5807, Test: 0.5966


  2%|▏         | 2/100 [02:20<1:55:23, 70.65s/it]

Loss: 0.3783, Train: 0.6549, Val: 0.5993, Test: 0.6208


...

Loss: 0.2727, Train: 0.8214, Val: 0.7801, Test: 0.7495


 99%|█████████▉| 99/100 [2:25:02<01:54, 114.11s/it]

Loss: 0.2717, Train: 0.8200, Val: 0.7616, Test: 0.7380


100%|██████████| 100/100 [2:26:54<00:00, 88.15s/it] 

Loss: 0.2723, Train: 0.8208, Val: 0.7634, Test: 0.7424
